In [2]:
'''
Author.      : Aditya Jain
Date Started : 17th March, 2021
About        : This script scraps data from iNaturalist for moth species in Quebec, Canada
'''

!pip install pyinaturalist
!pip install ipyleaflet

     |████████████████████████████████| 61kB 4.5MB/s  eta 0:00:01
     |████████████████████████████████| 61kB 4.3MB/s 
     |████████████████████████████████| 51kB 3.5MB/s 
     |████████████████████████████████| 3.2MB 13.3MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
     |████████████████████████████████| 3.3MB 11.9MB/s 


In [3]:
from pyinaturalist.node_api import get_geojson_observations, get_taxa, get_places_nearby, get_places_by_id, get_observations, get_taxa_by_id, get_all_observations
# from pyinaturalist.rest_api import get_observations
from ipyleaflet import Map
import json
import math
from bson import json_util
from datetime import date
from tqdm import tqdm
import csv
import numpy as np
import pandas as pd

# iNat place and species IDs
CANADA_ID  = 6712
QUEBEC_ID  = 13336
LEPIDOP_ID = 47157

CUR_DATE   = date.today()
DATA_DIR   = "/content/drive/My Drive/Data/"



## 1. Fetch and Save json data from iNat
##### a) The below snippet finds per-page results

In [ ]:
# getting data for lepidopterans spotted in Quebec
quebec_res = get_observations(
    taxon_id=LEPIDOP_ID,
    photos=True,
    geo=True,
    place_id=QUEBEC_ID,
    quality_grade='research'
)

# getting data for lepidopterans spotted in Canada
canada_res = get_observations(
    taxon_id=LEPIDOP_ID,
    photos=True,
    geo=True,
    place_id=CANADA_ID,
    quality_grade='research'
)

total_quebec_points = quebec_res['total_results']

print("No. of lepidopterans spotted in Quebec: ", quebec_res['total_results'])
print("No. of lepidopterans spotted in Canada: ", canada_res['total_results'])

# saving Quebec data in json format
# with open('quebec_full-' + str(CUR_DATE) + '.json', 'w') as f:
#     json.dump(quebec_res, f, default=json_util.default)

No. of lepidopterans spotted in Quebec:  46420
No. of lepidopterans spotted in Canada:  420430


##### b) [Under Work] The get_all_observations API is troublesome, so manually going through all the pages and fetching the data

In [ ]:
max_per_page = 200                                  # no. of entries to display per page
no_of_pages  = math.ceil(total_quebec_points/200)   # no. of pages
iNat_dict    = {}                                   # dictionary to store lepidopteran species on iNat

# just for vizualisation
lat_list     = []
lon_list     = []

for i in tqdm(range(1,no_of_pages+1)):
  data = get_observations(
    taxon_id=LEPIDOP_ID,
    photos=True,
    geo=True,
    place_id=QUEBEC_ID,
    quality_grade='research',
    page=i,
    per_page=max_per_page,
    order_by='observed_on')
  
  data = data['results']
  
  for item in data:
    taxa_name = item['taxon']['name']
    iNat_ID   = item['taxon']['id']
    
    if iNat_ID not in iNat_dict.keys():
      iNat_dict[iNat_ID] = [taxa_name, 1]
    else:
      iNat_dict[iNat_ID][1] += 1

    lat_list.append(item['location'][0])
    lon_list.append(item['location'][1])


##### c) The below code snippet fetches a more concise data for lepidopterans in quebec

In [ ]:
quebec_res = get_geojson_observations(
    taxon_id=LEPIDOP_ID,
    photos=True,
    geo=True,
    place_id=QUEBEC_ID,
    quality_grade='research'
)

with open('quebec_full-' + 'geojson' + '.json', 'w') as f:
    json.dump(quebec_res, f, default=json_util.default)

data = quebec_res["features"]

iNat_dict    = {}                                   # dictionary to store lepidopteran species on iNat
lat_list     = []
lon_list     = []

for item in data:  
  taxa_name = item["properties"]["taxon_name"]
  iNat_taxonID   = item["properties"]["taxon_id"]
    
  if iNat_taxonID not in iNat_dict.keys():
    iNat_dict[iNat_taxonID] = [taxa_name, 1]
  else:
    iNat_dict[iNat_taxonID][1] += 1

  lat_list.append(item["geometry"]["coordinates"][1])
  lon_list.append(item["geometry"]["coordinates"][0])

print('Total species count:', len(iNat_dict.keys()))

## 2. Load json file

Loading quebec lepidopterans list

In [ ]:
filename  = "quebec_full-geojson.json"
path      = DATA_DIR + filename
f         = open(path,)
data      = json.load(f)['features']
iNat_dict = {}                        # dictionary to store lepidopteran species on iNat


for item in data:  
  taxa_name = item["properties"]["taxon_name"]
  iNat_taxonID   = item["properties"]["taxon_id"]
    
  if iNat_taxonID not in iNat_dict.keys():
    iNat_dict[iNat_taxonID] = [taxa_name, 1]
  else:
    iNat_dict[iNat_taxonID][1] += 1


Once the data is loaded, the species list is saved in a csv with their counts

In [ ]:
species_list = []

for id, value in iNat_dict.items():
  species_list.append([id, value[0], value[1]])

species_list = np.array(species_list, dtype=np.dtype(object))
species_list = species_list[species_list[:,2].argsort()[::-1]]
field_names  = np.array([['iNat_TaxonID', 'Scientific_Name', 'Total_Observations_in_Quebec']])
species_list = np.concatenate((field_names, species_list))
np.savetxt('Quebec-Lepidop-Species_iNat.csv', species_list, delimiter=",", fmt='%s')

#### Visualizing the observations geo-spatially

In [ ]:
import plotly.express as px

fig = px.scatter_geo(lat=lat_list,
                     lon=lon_list,
                     scope='north america',
                     title="Geo-locations of Lepidopterans spotted in Quebec"
                     )
fig.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.hist(hist, bins=len(hist))
plt.title("Histogram")
plt.show()

In [ ]:
hist = []
for key in iNat_dict.keys():
  hist.append(iNat_dict[key][1])

print(hist)

[47, 5, 14, 138, 26, 104, 175, 99, 45, 6, 99, 75, 214, 104, 30, 24, 158, 260, 13, 48, 71, 11, 165, 15, 100, 78, 101, 40, 25, 37, 84, 22, 18, 10, 18, 46, 318, 31, 325, 104, 112, 163, 23, 36, 8, 10, 13, 42, 114, 7, 11, 15, 108, 145, 18, 20, 64, 55, 20, 19, 204, 25, 2, 37, 123, 50, 25, 18, 6, 65, 81, 4, 206, 21, 18, 238, 6, 55, 14, 59, 151, 8, 28, 47, 4, 4, 5, 20, 30, 60, 72, 26, 18, 140, 8, 34, 281, 5, 12, 56, 6, 6, 235, 8, 115, 170, 28, 168, 9, 140, 5, 38, 5, 5, 13, 31, 198, 69, 121, 420, 12, 13, 18, 43, 134, 16, 95, 17, 177, 2, 10, 3, 36, 20, 1, 29, 37, 92, 38, 30, 7, 25, 1, 7, 30, 19, 40, 11, 21, 25, 1, 68, 15, 80, 170, 19, 44, 28, 7, 114, 320, 56, 186, 295, 59, 161, 23, 17, 102, 48, 26, 9, 6, 20, 8, 25, 28, 94, 30, 101, 25, 41, 4, 8, 31, 130, 242, 63, 48, 39, 50, 87, 76, 2, 19, 10, 24, 30, 9, 39, 78, 80, 9, 45, 426, 24, 19, 52, 12, 41, 20, 4, 6, 107, 2, 6, 95, 21, 44, 71, 49, 54, 38, 62, 143, 158, 42, 27, 99, 138, 30, 31, 11, 163, 13, 9, 79, 9, 47, 36, 41, 2, 52, 4, 151, 41, 84, 22, 

## 3. Reading moth species list (shared by Maxim)
First reading the file, getting the relevant fields, cleaning it and generating a new csv file

In [ ]:
filename  = 'quebec_mothlist.xlsx'
file_path = DATA_DIR + filename
file_path = 'listB_Quebec_Pohl2018.xlsx'
data      = pd.read_excel(file_path, sheet_name=None)
data      = data['Pohl et al 2018 Qc moths']
data      = data[['SP NO.', 'superfamily', 'family', 'subfamily', 'val genus', 'name', 'FULL NAME']]

# data.to_csv('quebec_moth_v1.csv')   # this contains relevant columns but some empty rows

# removing empty entries by checking a specific column
for row in data.index:
  if pd.isnull(data['family'][row]):
    data.drop(row, inplace=True)

print(data)
data.to_csv('quebec_moth_v2.csv')   # this contains relevant columns along with removed empty rows

      SP NO.  ...                                   FULL NAME
0     010001  ...         Epimartyria auricrinella Walsingham
1     070001  ...  Dyseriocrania griseocapitella (Walsingham)
2     070003  ...        Eriocrania semipurpurella (Stephens)
3        NaN  ...                                         NaN
4     070004  ...                  Eriocrania breviapex Davis
...      ...  ...                                         ...
3502  933680  ...                 Abagrotis alternata (Grote)
3503  933682  ...                Abagrotis forbesi (Benjamin)
3504  933683  ...             Abagrotis brunneipennis (Grote)
3505  933685  ...                    Abagrotis cupida (Grote)
3506  933688  ...           Abagrotis anchocelioides (Guenée)

[3507 rows x 7 columns]
       SP NO.  ...                                   FULL NAME
0      010001  ...         Epimartyria auricrinella Walsingham
1      070001  ...  Dyseriocrania griseocapitella (Walsingham)
2      070003  ...        Eriocrania semip

## 4. Reading Vermont moth species list (shared by KP)
First reading the file, getting the relevant fields, cleaning it and generating a new csv file

In [9]:
filename  = 'listC_Vermont_29March2021.xlsx'
file_path = DATA_DIR + filename
data      = pd.read_excel(file_path, sheet_name=None)
data      = data['Sheet2']
data      = data[['Hodges_No', 'P3no', 'group', 'superfamily', 'family', 
                  'subfamily', 'genus', 'specificEpithet', 'taxonRank',
                  'scientificName']]

# removing empty entries by checking a specific column
for row in data.index:
  if pd.isnull(data['scientificName'][row]):
    data.drop(row, inplace=True)

print(data)

data.to_csv('listC_Vermont_29March2021.csv')

      Hodges_No      P3no  ... taxonRank                 scientificName
0           1.0   10001.0  ...   species       Epimartyria auricrinella
1           3.0   70001.0  ...   species  Dyseriocrania griseocapitella
2           5.0   70003.0  ...   species      Eriocrania semipurpurella
3          31.0  110011.0  ...   species        Korscheltellus gracilis
4          18.0  110016.0  ...   species    Sthenopis argenteomaculatus
...         ...       ...  ...       ...                            ...
1935    11043.0  933685.0  ...   species               Abagrotis cupida
1936    11045.0  933688.0  ...   species       Abagrotis anchocelioides
1937    10658.0   34151.0  ...   species               Agrotis stigmosa
1938     3412.2  621291.2  ...   species          Dichrorampha aeratana
1939     6405.0  910822.0  ...   species          Digrammia gnophosaria

[1940 rows x 10 columns]


## 5. Comparing iNat and Maxim's Quebec Moth species list
Comparing the two lists to find which species are common in the two lists and which are missing in either of them

In [ ]:
iNat_name      = 'Quebec-Lepidop-Species_iNat.csv'
quebec_name    = 'quebec_moth_v2.csv'
iNat_file      = DATA_DIR + iNat_name
quebec_file    = DATA_DIR + quebec_name

iNat_data      = pd.read_csv(iNat_file)
quebec_data    = pd.read_csv(moth_file)

iNat_species   = iNat_data['Scientific_Name']
quebec_species = quebec_data['val genus'] + ' ' + quebec_data['name']

#### A) In iNat, not in quebec list
The iNat list is of complete lepidoptera order, hence, it will also contain butterfly species. However, if there is any quebec moth species in iNat but not in maxim list, that would be interesting (because the latter is supposedly a superset of quebec moth species)

In [ ]:
list_not_in_Moth = []
for item in iNat_species.index:
  if iNat_species[item] not in list(quebec_species):
    list_not_in_Moth.append(iNat_species[item])

list_not_in_Moth = pd.DataFrame(list_not_in_Moth)
list_not_in_Moth.to_csv('IniNat_NotQuebec.csv') 

#### B) In quebec list, not in iNat
This code snippet will find out the species in quebec list which are not spotted on iNat

In [ ]:
list_not_in_iNat = []
for item in quebec_species.index:
  if quebec_species[item] not in list(iNat_species):
    list_not_in_iNat.append(quebec_species[item])

list_not_in_iNat = pd.DataFrame(list_not_in_iNat)
list_not_in_iNat.to_csv('InQuebec_NotiNat.csv')

## 5. Fetch Data
Saving images from iNat

In [ ]:
iNat_name      = 'Quebec-Lepidop-Species_iNat.csv'
iNat_file      = DATA_DIR + iNat_name

iNat_data      = pd.read_csv(iNat_file)
# print(iNat_data)

iNat_species   = list(iNat_data['iNat_TaxonID'])
# iNat_species   = list(iNat_data['Scientific_Name'])

print(iNat_species)

data = get_observations(
    # taxon_name=iNat_species[:2],
    taxon_id=iNat_species[0],
    photos=True,
    geo=True,
    geoprivacy='open',
    quality_grade='research')


[48662, 58583, 55641, 47802, 57463, 59675, 58523, 81663, 47919, 67691, 145659, 56832, 1063932, 55626, 143274, 48548, 120217, 81677, 143728, 127460, 119013, 154446, 49133, 81681, 219746, 130017, 211009, 120214, 121183, 60839, 229817, 205300, 143518, 142993, 47916, 58576, 49716, 208521, 337467, 81665, 81687, 81656, 205231, 58531, 127457, 736797, 223997, 143445, 198813, 84428, 201281, 47980, 153076, 233010, 154582, 128242, 58586, 119492, 1081323, 224075, 198812, 82379, 143118, 205197, 143867, 219124, 212435, 226649, 119953, 121236, 127133, 133447, 220826, 63128, 81657, 154353, 211012, 68262, 143009, 144107, 81582, 232156, 212430, 84662, 126276, 58484, 143517, 143121, 82153, 81680, 56776, 118901, 177320, 143112, 233838, 213838, 217190, 131673, 205204, 215211, 227466, 217970, 82225, 82204, 211100, 213514, 143446, 129226, 213826, 119063, 217054, 122259, 121534, 54064, 48094, 228855, 117452, 218900, 81679, 124181, 606580, 233677, 321779, 226646, 129572, 205213, 211007, 204699, 219856, 231396,

In [ ]:
PER_PAGE  = 200
MAX_PAGES = 25   # allows for max 5000 observations to be fetched
import math

for id in iNat_species:
  
  print(id)
  data = get_observations(
    taxon_id=id,
    photos=True,
    geo=True,
    geoprivacy='open',
    quality_grade='research',)
  
  total_count = data['total_results']
  no_pages    = total_count/PER_PAGE
  no_pages    = math.floor(min(no_pages, MAX_PAGES))

  print(no_pages)

48662
25
58583
25
55641


Could not parse timestamp: Sun Jul 14 2019 14:59:23 GMT+0100 (GMT+1): Unknown string format: Sun Jul 14 2019 14:59:23 GMT+0100 (GMT+1)
Could not parse timestamp: Sat Aug 15 2020 10:50:23 GMT+0200 (GMT+2): Unknown string format: Sat Aug 15 2020 10:50:23 GMT+0200 (GMT+2)


25
47802


Could not parse timestamp: Sun Aug 08 2010 18:43:14 GMT+0400 (GMT+4): Unknown string format: Sun Aug 08 2010 18:43:14 GMT+0400 (GMT+4)


25
57463


...

Could not parse timestamp: 2019/07/12 1:30: AM EDT: Unknown string format: 2019/07/12 1:30: AM EDT


10
47980
25
153076
9
233010
4
154582
10
128242
14
58586
25
119492
5
1081323
18
224075
3
198812
25
82379
25
143118
13
205197
15
143867


Could not parse timestamp: 2019/07/12 1:30: AM EDT: Unknown string format: 2019/07/12 1:30: AM EDT


8
219124
1
212435


HTTPError: ignored

In [ ]:
print(data.keys())
print(data['page'])

dict_keys(['total_results', 'page', 'per_page', 'results'])
1


Getting iNat taxon ids for species in quebec list

In [ ]:
quebec_name    = 'quebec_moth_v2.csv'
quebec_file    = DATA_DIR + quebec_name
quebec_data    = pd.read_csv(quebec_file)

quebec_species = quebec_data['val genus'] + ' ' + quebec_data['name']
print(quebec_species)

0            Epimartyria auricrinella
1       Dyseriocrania griseocapitella
2           Eriocrania semipurpurella
3              Gazoryctra hyperboreus
4               Gazoryctra novigannus
                    ...              
2876                Abagrotis placida
2877              Abagrotis alternata
2878          Abagrotis brunneipennis
2879                 Abagrotis cupida
2880         Abagrotis anchocelioides
Length: 2881, dtype: object


In [ ]:
def get_iNat_id(species_list):
  '''
  given a list of species name, this function returns the unique iNat taxon ID
  '''
  id_list = []

  for item in species_list.index:
    species_name = species_list[item]
    data         = get_taxa(q=species_name, rank='species', per_page=200)

    if data['total_results']==0:
      id_list.append('None')
    else:
      id_list.append(data['results'][0]['id'])
      
  return id_list

In [ ]:
print(get_iNat_id(quebec_species))

### Miscellaneous - Not to Run

In [ ]:
print(len(hist))

1824


In [ ]:
id_list = []
for p in response['results']['community']:
  id_list.append(p['id'])
  print(p['name'], p['id'])

for p in response['results']['standard']:
  id_list.append(p['id'])
  print(p['name'], p['id'])

Great Lakes Basin 57637
New England-Acadian forests 128777
Northeast & Mid-Atlantic States 96557
Labrador 132047
Northeastern United States 64492
Gulf of St. Lawrence 134488
Maritimes Provinces of Canada 82171
Maritimes Provinces of Canada 82180
New England 52339
Experimenting with polygons 154600
North America 97394
United States 1
Canada 6712
Québec 13336
Newfoundland and Labrador 7289
New Brunswick 7587
Maine 17
Nord-du-Québec 49209
Division No. 10 49165
Sept-Rivières--Caniapiscau 49211


In [ ]:
QUEBEC_BOX = (62.350899, -64.170174, 45.083101, -78.449810)       # lat/long coordinates for bounding box for quebec
response   = get_places_nearby(*QUEBEC_BOX)
print(response)

In [ ]:
a = {
    'A': [0, 'Aditya'],
    'B': [1, 'Suman']
}

print(a)

{'A': [0, 'Aditya'], 'B': [1, 'Suman']}


In [ ]:
a['B'][0] += 1
print(a)


{'A': [0, 'Aditya'], 'B': [2, 'Suman']}
